---
title: "Overschrijdingsfrequentielijnen"
execute:
  eval: false
  echo: true
  output: true
  warning: false
  error: false
---

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import pydra_core as pydra

Als demonstratie worden overschrijdingsfrequentielijnen voor de Borselle (Westerschelde/Kust) bepaald. Hieronder definiëren we de locatie van de database.

In [2]:
DB_PATH = "data/WBI2017_Westerschelde_30-4_v03.sqlite"

<h3>HRDatabase</h3>

Met de HR database maken we een <i>HRDatabase</i> aan. Dit object beheert alle locaties in de HR database en helpt om batch berekeningen te maken.

In [3]:
hrdatabase = pydra.HRDatabase(DB_PATH)

Eén van de functies van een <i>HRDatabase</i> object is het uitlezen van de namen van alle uitvoerpunten.

In [4]:
hrdatabase.get_location_names()

<h3>Settings</h3>

Met het <i>HRDatabase</i> maken we een <i>Settings</i> object aan voor het HR-uitvoerpunt 'WS_1_30-4_dk_00002'. Het <i>Settings</i> object is voor Pydra hetzelfde als het 'invoer.hyd' bestand voor Hydra-NL is.

In [5]:
settings = hrdatabase.get_settings("WS_1_30-4_dk_00002")

De settings zijn gebaseerd op wat standaard in Hydra-NL (versie 2.8.2) zit. Het is mogelijk om alle settings uit te lezen door het <i>Settings</i> object te printen.

In [6]:
print(settings)

Voordat berekeningen worden gedaan is het mogelijk om het <i>Settings</i> object aan te passen:

In [7]:
print("Origineel:", settings.m_max)
settings.m_max = 10.0
print("Aangepast:", settings.m_max)

<h3>Location</h3>

Wanneer je tevreden bent met de settings in het <i>Settings</i> object, kan een <i>Location</i> object worden aangemaakt. Wanneer een location wordt aangemaakt lees Pydra de statistiek in en worden belastingsmodellen gegenereert. De statistiek wordt ingelezen vanuit de <i>data/statistics</i> map in de package. De belastingsmodellen worden geïnitieerd op basis van de eerder gekoppelde HR database in het <i>HRDatabase</i> object.

In [8]:
import warnings

warnings.filterwarnings("ignore")  # future warnings are annoying
location = hrdatabase.create_location(settings)
# TODO: deze error beter afvangen:

Nu het <i>Location</i> object is aangemaakt voor het HR uitvoerpunt 'WS_1_30-4_dk_00002', kan deze gebruikt worden in berekeningen.

<h3>Overschrijdingsfrequentielijn voor één uitvoerpunt</h3>

Pydra heeft verschillende rekenmodules (objecten). Hieronder gebruiken we de <i>ExceedanceFrequencyLine</i> module om de overschrijdingsfrequentielijn te berekenen. Voordat we dat kunnen doen moet eerst een rekenobject worden aangemaakt, hieronder <i>fl</i>. Het rekenobject omvat de instellingen voor de berekening, bijvoorbeeld voor welke variabel een frequentielijn moet worden berekend (hieronder waterstand (h)). Daarnaast zijn er nog een aantal optionele opties, zoals of er met modelonzekerheden moet worden gerekend (standaard : True), de stapgrootte van de frequentielijn (standaard : 0.05) en een aangepast bereik van waterstanden voor de frequentielijn (standaard tussen de 1ste en 99de percentiel vanuit de HR database).

In [9]:
fl = pydra.ExceedanceFrequencyLine("h", model_uncertainty=False)

Vervolgens kunnen we met het rekenobject <i>fl</i> een overschrijdingsfrequentielijn bepalen voor HR uitvoerpunt 'WS_1_30-4_dk_00002' door de <i>calculate()</i> functie aan te roepen van het rekenobject en het <i>location</i> object als argument mee te geven.

In [10]:
frequency_line = fl.calculate(location)

Dit geeft een overschrijdingsfrequentielijn <i>frequency_line</i>. Het is een object waarin de berekende gegevens (waterstanden en overschrijdingsfrequenties) worden opgeslagen. Als je het object print, zie je wat er allemaal in zit.

In [11]:
print(frequency_line)

Hieronder is een plot van de berekende overschrijdingsfrequentielijn gegeven.

In [12]:
tt = np.array([10, 30, 100, 300, 1_000, 3_000, 10_000, 30_000, 100_000])

plt.figure(figsize=[8, 5])
plt.grid()
plt.plot(tt, frequency_line.interpolate_exceedance_probability(1 / tt), label="Pydra")
plt.legend(loc="upper left")
plt.title("Overschrijdingsfrequentielijn waterstand (WS_1_30-4_dk_00002)")
plt.xscale("log")
plt.xticks([10, 100, 1_000, 10_000, 100_000])
plt.xlabel("Terugkeertijd [jaar]")
plt.xlim(10, 100_000)
plt.ylabel("Waterstand [NAP+m]")
plt.ylim(3.5, 7.0)
plt.show()

We hebben dezelfde berekening gemaakt met Hydra-NL, wat de onderstaande vergelijking geeft.

In [13]:
hydranl_h = [3.998, 4.28, 4.592, 4.879, 5.198, 5.494, 5.828, 6.141, 6.497]

plt.figure(figsize=[8, 5])
plt.grid()
plt.plot(tt, frequency_line.interpolate_exceedance_probability(1 / tt), label="Pydra")
plt.plot(tt, hydranl_h, ":", label="Hydra-NL")
plt.legend(loc="upper left")
plt.title("Overschrijdingsfrequentielijn waterstand (WS_1_30-4_dk_00002)")
plt.xscale("log")
plt.xticks([1, 10, 100, 1_000, 10_000, 100_000, 1_000_000])
plt.xlabel("Terugkeertijd [jaar]")
plt.xlim(10, 100_000)
plt.ylabel("Waterstand [NAP+m]")
plt.ylim(3.5, 7.0)
plt.show()

<h3>Overschrijdingsfrequentielijn voor een heel traject</h3>

Het is ook mogelijk om overschrijdingsfrequentielijnen voor een heel traject te genereren. Laten we weer een rekenobject <i>fl</i> aanmaken. Merk op dat we  ten opzichte van hierboven dit maal <i>model_uncertainty = False</i> hebben toegevoegd. Dit betekent dat we nu <u>zonder</u> modelonzekerheden rekenen.

In [14]:
fl = pydra.ExceedanceFrequencyLine("hs", model_uncertainty=False)

Door het <i>HRDatabase</i> object '<i>hrdatabase</i>' mee te geven als argument in de <i>calculate()</i> functie, bepaald Pydra de overschrijdingsfrequentielijn voor <u>alle</u> uitvoerpunten in de HR database. Voor locaties die al zijn gedefinieerd, zoals 'WS_1_30-4_dk_00002', wordt het eerder gedefinieerde <i>Location</i> object gebruikt. Voor uitvoerpunten die nog niet zijn gedefinieerd creëert Pydra automatisch een <i>Location</i> object met de standaardwaarden voor het water systeem (net zoals Hydra-NL een 'invoer.hyd' aanmaakt in de beoordelingsmodus).

Merk op dat dus in dit geval voor uitvoerpunt 'WS_1_30-4_dk_00002' met een <i>mmax</i> van NAP+10,0m wordt gerekend omdat we die eerder hebben aangepast. Voor de overige locaties wordt met de standaard NAP+9,0m gerekend.

In [15]:
freq_lines = fl.calculate(hrdatabase)
# TODO: deze errors beter afvangen:

Omdat we nu een heel traject doorrekenen, zal <i>calculate()</i> niet één frequentielijn returnen. In plaats daarvan wordt een dictionary gereturnd met als key de naam van het uitvoerpunt en als value het <i>FrequencyLine</i> resultaat object.

In [16]:
print(freq_lines["WS_1_30-4_dk_00006"])

Een overzicht van alle resultaten:

In [17]:
plt.figure(figsize=[8, 5])
plt.grid()

for _loc in freq_lines:
    plt.plot(
        1 / freq_lines[_loc].exceedance_frequency, freq_lines[_loc].level, label=_loc
    )

plt.legend(loc="upper left")
plt.title(
    "Overschrijdingsfrequentielijn significante golfhoogte zonder modelonzekerheid"
)
plt.xscale("log")
plt.xticks([1, 10, 100, 1_000, 10_000, 100_000])
plt.xlabel("Terugkeertijd [jaar]")
plt.xlim(1, 100_000)
plt.ylabel("Significante golfhoogte [m]")
plt.ylim(0, None)
plt.show()